# usALEX - Corrections - Direct excitation physical parameter

> *This notebook estimates direct excitation coefficient $d_{exT}$ from μs-ALEX data.*

## Aim

### What is already computed?

We previously fitted the **leakage** and **gamma** coefficient from the RAW PR values for the 5 measurements. We also fitted the
direct excitation coefficient expressed (`dir_ex_aa`) as a function of the A-signal during A-excitation (`naa`). In symbols, `dire_ex_aa` is defined as:

$$ d_{exAA} = \frac{n_{a}}{n_{aa}}$$

for a A-only population.

### What we want to compute?

An alternative defition of the direct excitation coefficient (`dir_ex_t`, $d_{exT}$) relies only on physical properties but, differently from `dir_ex_aa`, it cannot be expressed as a function of μs-ALEX quatitites.

The definition of $d_{exT}$ is:

$$ d_{exT} = \frac{\sigma_{A532}}{\sigma_{A532}}$$

where $\sigma_{A532}$ and $\sigma_{D532}$ are the absorption cross-sections of the Acceptor and Donor dye at 532nm.

Once dyes and excitation wavelength are fixed this coefficien is also fixed. Moreover, since $d_{exT}$ is independent from μs-ALEX quantities, it can also used to correct the non-ALEX multi-spot measurements.

### How we compute it?

Here, we correct the RAW PR values using the formula for leakage + direct excitation (`dir_ex_t`) + gamma correction. Since we already estimated leakage and gamma,  we fit `dir_ex_t` in order to match the corrected FRET efficiencies obtained from corrected FRET histograms. The fitted direct excitation (`dir_ex_t`) will be finally used to correct the multi-spot measurements.

## Library imports

In [ ]:
from __future__ import division
import os
import numpy as np
import pandas as pd
import lmfit

## Load data RAW PR data

In [ ]:
#bsearch_ph_sel = 'AND-gate'
bsearch_ph_sel = 'Dex'

data_file = 'results/usALEX-5samples-PR-raw-%s.txt' % bsearch_ph_sel

These are the **RAW proximity ratios** for the 5 samples (only background correction, no leakage nor direct excitation):

In [ ]:
data_raw = pd.read_csv(data_file, sep="\s+").set_index('sample')
data_raw[['E_gauss_w', 'E_kde_w']]

## Load μs-ALEX corrections

In [ ]:
leakage_coeff_fname = 'results/usALEX - leakage coefficient DexDem.txt'
leakage = np.loadtxt(leakage_coeff_fname)

print('Leakage coefficient:', leakage)

In [ ]:
dir_ex_t_datasheet_fname = 'results/Dyes - ATT0647N-ATTO550 abs X-section ratio at 532nm.txt'
dir_ex_t_datasheet = np.loadtxt(dir_ex_t_datasheet_fname)

print('Direct excitation (dir_ex_t) from datasheet:', dir_ex_t_datasheet)

In [ ]:
gamma_coeff_fname = 'results/usALEX - gamma factor - all-ph.txt'
gamma = np.loadtxt(gamma_coeff_fname)

print('Gamma factor:', gamma)

## Load E data from corrected FRET bursts

And these are the **FRET efficiencies** fitted from corrected histograms for the same 5 samples: 

In [ ]:
data_file = 'results/usALEX-5samples-E-corrected-all-ph.txt'
data_corr = pd.read_csv(data_file, sep="\s+").set_index('sample')
data_corr[['E_gauss_w', 'E_kde_w']]

### Proximity Ratio correction formula

The expression to convert RAW PR values to corrected FRET efficiencies:

$$ E = \frac{E_{R} \left(L_{k} + d_{exT} \gamma + 1\right) - L_{k} - d_{exT} \gamma}{E_{R} \left(L_{k} - \gamma + 1\right) - L_{k} + \gamma}$$

*See [Derivation of FRET and S correction formulas](Derivation of FRET and S correction formulas.ipynb) for the derivation*.

We load [FRETBursts software](https://github.com/tritemio/FRETBursts/) that provides this function (as `fretmath.correct_E_gamma_leak_dir`):

In [ ]:
from fretbursts import *

## Comparison of datasheet-based *vs* μs-ALEX-based direct excitation

Using [datasheet values](dyes/Absorption ATTO550-ATTO647N.ipynb) for $d_{exT}$ we the following FRET efficiencies:

In [ ]:
E_datasheet = fretmath.correct_E_gamma_leak_dir(data_raw.E_kde_w, 
                                                leakage=leakage, 
                                                dir_ex_t=dir_ex_t_datasheet,
                                                gamma=gamma)*100
E_datasheet

In [ ]:
out = data_corr[['E_kde_w']].copy()*100
out.columns = ['E_alex']
out['E_datasheet'] = E_datasheet
out

In [ ]:
out.plot(alpha=0.4, lw=3, style=dict(E_alex='-ob', E_datasheet='-sr'));

> **NOTE:** The corrected FRET efficiencies using the datasheet and 
> μs-ALEX-based direct excitation do not match well.

## Fitting of physical direct excitation coefficient $d_{exT}$

In [ ]:
import lmfit

In [ ]:
def residuals_absolute(params, E_raw, E_ref):
    dir_ex_t = params['dir_ex_t'].value
    return E_ref - fretmath.correct_E_gamma_leak_dir(E_raw, 
                                                     leakage=leakage, 
                                                     gamma=gamma, 
                                                     dir_ex_t=dir_ex_t)

In [ ]:
def residuals_relative(params, E_raw, E_ref):
    dir_ex_t = params['dir_ex_t'].value
    return (E_ref - fretmath.correct_E_gamma_leak_dir(E_raw, 
                                                      leakage=leakage, 
                                                      gamma=gamma, 
                                                      dir_ex_t=dir_ex_t))/E_ref

In [ ]:
params = lmfit.Parameters()
params.add('dir_ex_t', value=0.05) 

In [ ]:
m = lmfit.minimize(residuals_absolute, params, args=(data_raw.E_kde_w, data_corr.E_kde_w))
lmfit.report_fit(m.params, show_correl=False)

In [ ]:
m = lmfit.minimize(residuals_relative, params, args=(data_raw.E_kde_w, data_corr.E_kde_w))
lmfit.report_fit(m.params, show_correl=False)

> **NOTE:** The fitted `dir_ex_t` is 4.5% as opposed to 10.6% as expected from the [absorption spectra of ATTO550 and ATTO647](dyes/Absorption ATTO550-ATTO647N.ipynb) at 532nm.

In [ ]:
m.params['dir_ex_t'].value

In [ ]:
with open('results/usALEX - direct excitation coefficient dir_ex_t.txt', 'w') as f:
    f.write(str(m.params['dir_ex_t'].value))

## Use `dir_ex_t` ($d_{exT}$) to obtain corrected $E$

In [ ]:
data_raw

In [ ]:
E_corr_pr = fretmath.correct_E_gamma_leak_dir(data_raw.E_kde_w, 
                                              leakage=leakage, 
                                              dir_ex_t=m.params['dir_ex_t'].value,
                                              gamma=gamma)*100
E_corr_pr

In [ ]:
out['E_corr_pr'] = E_corr_pr
out

In [ ]:
out['E_alex']

In [ ]:
x = [int(idx[:-1]) for idx in out.index]
plot(x, out['E_alex'])
plot(x, out['E_corr_pr'])
(out['E_alex'] - out['E_corr_pr']).abs().max()

In [ ]:
(out['E_alex'] - out['E_corr_pr'])

> **NOTE:** Fitting $d_{exT}$ to match $E$ from corrected hsitograms with $E$ from PR correction formula produces a max difference of 1% for the 12d sample. The match is well below the fitting accuracy (> 2%).